In [1]:
# Imports
import numpy as np
import os
from osgeo import gdal

gdal.UseExceptions()

In [6]:
# Config Values
patch_size = 200
step_size = 200
DESTINATION = 'Users/freyachay/Documents/Stanford/junior/SALT_MARSHES/training/'
INPUT_DIR = '/Users/freyachay/Documents/Stanford/junior/SALT_MARSHES/git/'
INPUT_NAMES = ['bayArea_mosaic.tif']

In [3]:
# Credit to nshaud 
# (https://github.com/nshaud/DeepNetsForEO/blob/master/notebooks/Image%20extraction.ipynb) for general outline of making patches.
def sliding_window(image, patch_size, step_size):
    """Extract chips using a sliding window
    
    Args:
        image (numpy array): The image to be processed.
        stride (int): The sliding window stride.
        patch_size(int, int, optional): The patch size.

    Returns:
        list: list of patches with patch_size dimensions
    """
    patches = []
    discarded = 0
    for i in range(0, image.shape[0], step_size):
        for j in range(0, image.shape[1], step_size):
            new_patch = image[:, i:i+patch_size, j:j+patch_size]
            if new_patch.shape[1] == new_patch.shape[2] == patch_size:
                patches.append(new_patch)
            else:
                discarded += 1
    return patches, discarded

In [4]:
# takes image in numpy arr form
def make_patches(image, trans, proj):
    patches, num_discarded = sliding_window(arr, patch_size, step_size)
    bands, rows, cols = patches[0].shape
    
    for i in range(len(patches)):
        outfile = "tile_"+ str(i) + ".tif"
        print outfile
        patch = patches[i]
        # Create the file, using the information from the original file
        outdriver = gdal.GetDriverByName("GTiff")
        outdata   = outdriver.Create(str(outfile), rows, cols, bands, gdal.GDT_Int16) #### Might need to be int16

        # Georeference the image
        outdata.SetGeoTransform(trans)

        # Write projection information
        outdata.SetProjection(proj)

        # Write the array to the file, which is the original array in this example
        for i in range(1,bands+1):
            outdata.GetRasterBand(i).WriteArray(patch[i-1], 0, 0)
            outdata.GetRasterBand(i).FlushCache()

In [7]:
# Get image as array

mosaic_path = INPUT_DIR + "bayArea_mosaic.tif"
# labels_path = INPUT_DIR + "bayArea_mosaic_labels.tif"

try:
    data = gdal.Open(mosaic_path)
except RuntimeError, e:
    print 'Unable to open ' + mosaic_path +':'
    print e
trans = data.GetGeoTransform()
proj = data.GetProjection()
bands_data = []
for b in range(1, data.RasterCount+1):
    band = data.GetRasterBand(b)
    bands_data.append(band.ReadAsArray())

bands_data = np.dstack(bands_data)
rows, cols, n_bands = bands_data.shape
print (rows,cols, n_bands)

(24200, 19400, 5)


In [8]:
# Credit to  Carlos De La Torre
# http://www.machinalis.com/blog/python-for-geospatial-data-processing/?utm_content=bufferf65db&amp;utm_medium=social&amp;utm_source=twitter.com&amp;utm_campaign=buffer

def create_mask_from_vector(vector_data_path, cols, rows, geo_transform,
                            projection, target_value=1):
    """Rasterize the given vector (wrapper for gdal.RasterizeLayer)."""
    data_source = gdal.OpenEx(vector_data_path, gdal.OF_VECTOR)
    layer = data_source.GetLayer(0)
    driver = gdal.GetDriverByName('MEM')  # In memory dataset
    target_ds = driver.Create('', cols, rows, 1, gdal.GDT_UInt16)
    target_ds.SetGeoTransform(geo_transform)
    target_ds.SetProjection(projection)
    gdal.RasterizeLayer(target_ds, [1], layer, burn_values=[target_value])
    return target_ds


def vectors_to_raster(path, rows, cols, geo_transform, projection):
    """Rasterize the vectors in the given directory in a single image."""
    labeled_pixels = np.zeros((rows, cols))
    ds = create_mask_from_vector(path, cols, rows, geo_transform,
                                 projection, target_value=1)
    band = ds.GetRasterBand(1)
    labeled_pixels += band.ReadAsArray()
    ds = None
    return labeled_pixels

In [9]:
# Get labels as array 
shapefile = INPUT_DIR + "bayArea_mosaic_label_clipped.shp"
labeled_pixels = vectors_to_raster(shapefile, rows, cols, trans,proj)

--------------------------------------------------


EXPERIMENTING BELOW WITH SIMPLE CLASSIFICATION
TODO: Implement sharding + saving as tfrecords


--------------------------------------------------

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [10]:
# Separate marsh and other pixels
is_marsh = (labeled_pixels !=0)
marsh_data = bands_data[is_marsh]
other_data = bands_data[~is_marsh]

In [11]:
# Create train and validation pixel sets (80% / 20% split)

marsh_train_indices = np.random.choice(np.arange(marsh_data.shape[0]), marsh_data.shape[0] * 0.8, replace=False)
marsh_train = marsh_data[marsh_train_indices, :]
marsh_val = np.delete(marsh_data, marsh_train_indices, axis=0)

other_train_indices = np.random.choice(np.arange(other_data.shape[0]), other_data.shape[0] * 0.8, replace=False)
other_train = other_data[other_train_indices, :]
other_val = np.delete(other_data, other_train_indices, axis=0)

train_data = np.vstack((marsh_train, other_train))
train_labels = np.concatenate((np.ones(marsh_train.shape[0]), np.zeros(other_train.shape[0])))

val_data = np.vstack((marsh_val, other_val))
val_labels = np.concatenate((np.ones(marsh_val.shape[0]), np.zeros(other_val.shape[0])))

print(train_data.shape)
print(train_labels.shape)
print(train_labels)

print(val_data.shape)
print(val_labels.shape)
print(val_labels)


/Users/freyachay/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  app.launch_new_instance()
/Users/freyachay/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


(375583999, 5)
(375583999,)
[ 1.  1.  1. ...,  0.  0.  0.]
(93896001, 5)
(93896001,)
[ 1.  1.  1. ...,  0.  0.  0.]


In [35]:
# Downsize for easy experimentation
num_train = train_labels.size
ex_train_indices = np.random.choice(np.arange(num_train), num_train * 0.00001, replace=False)
ex_train_data = train_data[ex_train_indices,:]
ex_train_labels = train_labels[ex_train_indices]

num_val = val_labels.size
ex_val_indices = np.random.choice(np.arange(num_val), num_val * 0.00001, replace=False)
ex_val_data = val_data[ex_val_indices,:]
ex_val_labels = val_labels[ex_val_indices]

/Users/freyachay/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  app.launch_new_instance()
/Users/freyachay/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [58]:
print ("% marsh train pixels: " + str(100 * float(marsh_train_indices.size) /(marsh_train_indices.size + other_train_indices.size)))
print ("% marsh val pixels: "+ str(100 * (float(marsh_val.shape[0]) /(marsh_val.shape[0] + other_val.shape[0]))))
print ("")

print(ex_train_data.shape)
print(ex_train_labels.shape)
print(ex_train_labels)

print(ex_val_data.shape)
print(ex_val_labels.shape)
# print(ex_val_labels)
print("")

print ("% marsh ex_train pixels: " + str(100* sum(ex_train_labels)/ex_train_labels.size))
print ("% marsh ex_val pixels: "+ str(100* sum(ex_val_labels)/ex_val_labels.size))
print ("")

% marsh train pixels: 3.51331660431
% marsh val pixels: 3.51331682379

(3755, 5)
(3755,)
[ 0.  0.  0. ...,  0.  0.  0.]
(938, 5)
(938,)

% marsh ex_train pixels: 3.88814913449
% marsh ex_val pixels: 3.94456289979



In [ ]:
# root = "/Users/freyachay/Documents/Stanford/junior/SALT_MARSHES/baselineData/"
# np.savetxt( root + "train.csv", np.column_stack((train_labels,train_data)), delimiter=" ")
# np.savetxt( root + "val.csv", np.column_stack((val_labels,val_data)), delimiter=" ")

In [ ]:
# # Load data from CSV files

# train = numpy.loadtxt(root + "train.csv")
# train_label = train[:,0]
# train_data = train[:,1:]

# val = numpy.loadtxt(root + "val.csv")
# val_label = val[:,0]
# val_data = val[:,1:]

In [38]:
# Per pixel Random Forest Classifier
classifier = RandomForestClassifier(n_jobs=-1)
classifier.fit(ex_train_data, ex_train_labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=-1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [42]:
predicted_labels = classifier.predict(ex_val_data)

from sklearn import metrics

print("Confussion matrix:\n%s" %
      metrics.confusion_matrix(ex_val_labels, predicted_labels))

# print("Classification report:\n%s" %
#       metrics.classification_report(ex_val_labels, predicted_labels,
#                                     target_names=[]))
print("Classification accuracy: %f" %
      metrics.accuracy_score(ex_val_labels, predicted_labels))

Confussion matrix:
[[899   2]
 [ 29   8]]
Classification accuracy: 0.966951


In [44]:
# Per pixel Random Forest Classifier
classifier = RandomForestClassifier(n_jobs=-1, class_weight="balanced")
classifier.fit(ex_train_data, ex_train_labels)

predicted_labels = classifier.predict(ex_val_data)

from sklearn import metrics

print("Confusion matrix:\n%s" %
      metrics.confusion_matrix(ex_val_labels, predicted_labels))

# print("Classification report:\n%s" %
#       metrics.classification_report(ex_val_labels, predicted_labels,
#                                     target_names=[]))
print("Classification accuracy: %f" %
      metrics.accuracy_score(ex_val_labels, predicted_labels))

Confussion matrix:
[[898   3]
 [ 28   9]]
Classification accuracy: 0.966951


In [46]:
# Per pixel Random Forest Classifier
classifier = RandomForestClassifier(n_jobs=-1, class_weight="balanced_subsample")
classifier.fit(ex_train_data, ex_train_labels)

predicted_labels = classifier.predict(ex_val_data)
print("Confusion matrix:\n%s" %
      metrics.confusion_matrix(ex_val_labels, predicted_labels))

print("Classification accuracy: %f" %
      metrics.accuracy_score(ex_val_labels, predicted_labels))

Confussion matrix:
[[895   6]
 [ 28   9]]
Classification accuracy: 0.963753


In [55]:
from sklearn import svm
classifier = svm.SVC(gamma=2, C=5000.)
classifier.fit(ex_train_data, ex_train_labels)

predicted_labels = classifier.predict(ex_val_data)

print("Confusion matrix:\n%s" %
      metrics.confusion_matrix(ex_val_labels, predicted_labels))

print("Classification accuracy: %f" %
      metrics.accuracy_score(ex_val_labels, predicted_labels))

Confusion matrix:
[[901   0]
 [ 37   0]]
Classification accuracy: 0.960554


In [57]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

classifier = QuadraticDiscriminantAnalysis()
classifier.fit(ex_train_data, ex_train_labels)

predicted_labels = classifier.predict(ex_val_data)

print("Confusion matrix:\n%s" %
      metrics.confusion_matrix(ex_val_labels, predicted_labels))

print("Classification accuracy: %f" %
      metrics.accuracy_score(ex_val_labels, predicted_labels))

Confusion matrix:
[[856  45]
 [ 13  24]]
Classification accuracy: 0.938166
